In [3]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
for study in os.listdir("Studies/"):
    for orientation in os.listdir("Studies/"+study+"/"):
        for folder in os.listdir("Studies/"+study+"/"+orientation+"/"):
            if folder == "anno":
                df = pd.read_csv("Studies/"+study+"/"+orientation+"/"+folder+"/gTruth.csv").dropna().reset_index().drop(columns="index")
                df["PixelLabelData"] = df["PixelLabelData"].map(lambda path: "/".join(["Studies"]+path.split("/")[-5:]))
                df["OriginalData"] = ""
                try:
                    os.mkdir("Studies/"+study+"/"+orientation+"/"+folder+"/OriginalData/")
                    vidcap = cv2.VideoCapture("Studies/"+study+"/"+orientation+"/clip/"+orientation+".mp4")
                    for i in range(len(df)):
                        name = df.at[i, 'PixelLabelData'].split("/")[-1]
                        time = round(float(df.at[i, 'Time'].split(" ")[0])*1000)
                        vidcap.set(cv2.CAP_PROP_POS_MSEC, time)
                        success,image = vidcap.read()
                        if success:
                            cv2.imwrite("Studies/"+study+"/"+orientation+"/"+folder+"/OriginalData/"+name, image)
                            df.at[i, "OriginalData"] = "Studies/"+study+"/"+orientation+"/"+folder+"/OriginalData/"+name
                    df = df.replace('', np.nan).dropna().reset_index().drop(columns="index")
                    df.to_csv("Studies/"+study+"/"+orientation+"/"+"data.csv", index=False)
                except:
                    print("folder done")

In [ ]:
finaldf = []
for study in os.listdir("Studies/"):
    for orientation in os.listdir("Studies/"+study+"/"):
        for fsobject in os.listdir("Studies/"+study+"/"+orientation+"/"):
            if fsobject == "data.csv":
                df = pd.read_csv("Studies/"+study+"/"+orientation+"/"+fsobject)
                finaldf.append(df)
finaldf = pd.concat(finaldf, ignore_index=True)
finaldf.to_csv("dataset.csv", index=False)

In [ ]:
dataset = pd.read_csv("dataset.csv")
try:
    os.mkdir("data/")    
    os.mkdir("data/train_frames/")
    os.mkdir("data/mask_frames/")
    os.mkdir("data/train_frames/train")
    os.mkdir("data/mask_frames/mask")
    for i in range(len(dataset)):
        image = cv2.imread(dataset.at[i, 'OriginalData'])
        mask = cv2.imread(dataset.at[i, 'PixelLabelData'])
        cv2.imwrite("data/train_frames/train/"+str(i)+".png", image)
        cv2.imwrite("data/mask_frames/mask/"+str(i)+".png", mask)
except:
    for i in range(len(dataset)):
        image = cv2.imread(dataset.at[i, 'OriginalData'])
        mask = cv2.imread(dataset.at[i, 'PixelLabelData'])
        cv2.imwrite("data/train/train/"+str(i)+".png", image)
        cv2.imwrite("data/mask/mask/"+str(i)+".png", mask)

In [4]:
dataset = pd.read_csv("dataset.csv")
image = cv2.imread(dataset.at[1, 'OriginalData'])
mask = cv2.imread(dataset.at[1, 'PixelLabelData'], cv2.IMREAD_GRAYSCALE)
(thresh, im_bw) = cv2.threshold(mask, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
cv2.imshow("bw",im_bw)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)# cv2.addWeighted(im_bw, alpha, output, 1 - alpha, 0, output)
# print(sum(sum(im_bw)))

TypeError: imshow() missing required argument 'mat' (pos 2)

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

In [ ]:
train_image_generator = train_datagen.flow_from_directory('data/train_frames/', batch_size=32)
train_mask_generator = train_datagen.flow_from_directory('data/mask_frames/', batch_size=32)

In [ ]:
train_generator = zip(train_image_generator, train_mask_generator)